In [30]:
intention = '''Create an easy/safe API to parse/hold/yield constructions of pydantic docx class objects.
In theory this can be generalized and abstracted since the classes are largely absracted. 
However since this is API based, I will just focus on solutions for THIS dictionary task. These can be expanded later, or just have other APIs added
Final objective is have a tool to pass complex commands/instructions, and have it yield a pandas dataframe of the results.
Intermediate steps may be yield a list of lists
I am not yet sure if there is benefit trying to have the whole dictionary in one notional object. 
I don't think so, since each operation is psuedo atomic, and any aggregate actions are intended to be done in something like pandas.
Eventually I would want a loop for pandas operations to easily feed updates to the data. 
'''
conclusion = '''create a pular dictionary specifc datastructure to handle the aggregate paragraphs. 
This will only be per lemma, 
as any aggregations from roots and relations of that sort can be managed in pandas and created by simple inheretance/reference
'''

In [31]:
from typing import Optional, Dict, List, Any, Union, Tuple
import pydantic
from pydantic import ValidationError, validator, root_validator, Field, constr
from pydantic_docx import Docx_Paragraph_and_Runs, read_docx #type:ignore
from pydantic_docx_processor import create_sized_dataframe, expand_dataframe #type:ignore
import re
import json
from itertools import compress, chain
from datetime import datetime
import pandas as pd
import numpy as np
from functools import partial

In [32]:
import pickle
with open('feature_Frames_and_Indexes.pkl', 'rb') as file:
    # Call load method to deserialze
    output = pickle.load(file, encoding='utf-8')
(
parsed_object_list, #:List[Tuple[int,Docx_Paragraph_and_Runs]]
parsed_object_lookup, #:Dict[int,Docx_Paragraph_and_Runs] = dict(parsed_object_list)
doc_para_count, #: int = int(parsed_to_dict['total_encountered_paragraphs']) #type: ignore
char_counts, #: Counter = parsed_to_dict['char_counts'] #type: ignore
rootFrame,#:pd.DataFrame
rootsubpieceFrame, #:pd.DataFrame
lemmaFrame, #:pd.DataFrame
nonentityParaFrame, #pd.DataFrame
cleanerOutcomesDf #pd.DataFrame
    ) = output
paratext_lookup = {k:v.interogate__para_text() for k,v in parsed_object_lookup.items()}

In [33]:
#first check the frame containing the success/failure boolean for the cleaner operation that was performed on the data in pydantic_docx ingestion
# print(cleanerOutcomesDf['cleaner_success_outcomes'].unique())
# print(cleanerOutcomesDf.groupby('cleaner_success_outcomes').count())
cleanerOutcomesDf.dropna(subset=['paragraphIndex'],inplace=True)
# print(cleanerOutcomesDf['cleaner_success_outcomes'].unique())
# print(cleanerOutcomesDf.groupby('cleaner_success_outcomes').count())
# print(cleanerOutcomesDf[cleanerOutcomesDf['cleaner_success_outcomes']==False])
cleanerfailedInds = list(cleanerOutcomesDf[cleanerOutcomesDf['cleaner_success_outcomes']==False].index)
print(cleanerfailedInds)


all_para_text_frame = pd.Series(paratext_lookup).to_frame()
all_para_text_frame = expand_dataframe(all_para_text_frame,doc_para_count)
spread = 10
window_of_inds = [list(range(i-spread,i+spread)) for i in cleanerfailedInds]

# print(len(nonentityParaFrame.index.intersection(cleanerfailedInds))) #>>> 7, so all were captured in the nonentityParaFrame
print(nonentityParaFrame.shape)
nonentityParaFrame.drop(cleanerfailedInds, axis = 'index', inplace=True)
print(nonentityParaFrame.shape)
# all_para_text_frame.loc[window_of_inds[6]]

#results: 

# [1982, 1983] #>

   # all_para_text_frame.loc[window_of_inds[0]]
      # 1981	NaN	
      #># 1982	\t\t\t\t\t\t	
      #># 1983	\t\t\t\t\t\t\t\t	
      # 1984	NaN	
      # 1985	NaN	
      # 1986	Ɓ	
# 5971 #>
   # all_para_text_frame.loc[window_of_inds[2]]
      #># 5971	\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t
      # 5972	FAKO Sonr	
      # 5973	fako ? n. o	 
# 9318 #>
   # all_para_text_frame.loc[window_of_inds[3]]
      # 9316	GUYAAJI guyaaji, guyaale, guyaali pl.- nguyk...	
      # 9317	NaN	
      #># 9318	\t	
      # 9319	NaN	
      # 9320	NaN	
      # 9321	H	
      # 9322	NaN	
# [9786, 9787] #>
   # all_para_text_frame.loc[window_of_inds[4]]
      # 9784	non-castrated male, stallion, bull	
      # 9785	mâle non castré(D), étalon(Z), taureau(Z)	
      #># 9786	\t\t	
      #># 9787	\t\t\t	
      # 9788	HALƁ-	
      # 9789	halɓude v.av DCZ CZ<FJ,M,NO> [haacude] cf...	
# 20565 #>
   # all_para_text_frame.loc[window_of_inds[6]]
      # 20564	NaN	20564
      #># 20565	\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t	20565
      # 20566	Ŋ	20566
      # 20567	NaN	20567	

# all values with a valid paragraph will have a value in 'paragraphIndex'. ie paragraphs that did not have a suppressed error during class assignment. By default these are lines with only '\n'.
#when parsing, the default was used, so any all whitespace paragraph will be nan in paragraphIndex.
#when these values are dropped, the boolean column (indicating the successful post-processing of already created pydantic_docx class objects) yields True/False values.
# 7 values were false, as shown above. Each was a line of only '\t'. Lines with only '\n' are not even read into the model.
# adding control for this into the class object validation requirements may be a good idea. But better yet would be to pass a config with regex or simple roots to discard lines of a certain type. I'll add that to #TODO
# we can safely drop these records.
# ...and therefor, all are "good" for the next steps

[1982, 1983, 5971, 9318, 9786, 9787, 20565]
(17241, 1)
(17234, 1)


The following merge routine merges together the parents of one level of the heirarchy with their children. 
The parents then have their values forward-filled / duplicated to account for their headship over the child entities
Each frame has an identity column with a boolean value indicating that index is the location of that entity.
These are not needed for propogating headship and or child relationships, and these will not be forward-filled.
As such, these will be removed from the entity frames before they are added to the entity heirarchy frame

Precedence is as follows:
root > subroot > lemmas > paragraphs

However, the child relationship is non-exclusive to direct 'parents', such that a parent 'entity' may have a dhild from any of the levels below them.
That said, the dictionary description and general structure would indicate paragraphs should only be direct children of Lemmas
>paragraphs being defined as any line without an 'entity' such as root, subroot, or lemma.
>any that are not would be a deviation. 
Therefor, any such instance needs to be captured and assessed, and should not be coerced into some other neighbors "family" by default, if at all.

As a result, our routine for creating hierarchys must allow for any possible/permitted parent be available to match any children.

So...

Subroots:      possible parents are:      Roots
Lemmas:        possible parents are:      Roots, Subroots
Paragraphs:    possible parents are:      Roots, Subroots, Lemmas

*Note: the term 'subroot' is sometimes described as 'root-subpiece', and these names may be understood interchangibly. The reason for the two names stems from two competing 'theories' or impressions of the nature of these entities. Initially I used the term subroot as a description of their subordinate position. The term subpiece has begun entering my vocabulary after discovering that many (if not all) of these are 'infixes', a core component of the Pular language. As a result any of these that are infixes are more than just a manual 'byte pair encoding' or common break point, but have specific meaning of their own.*

In [34]:
all_entity_identity_frame = rootFrame[['is']].merge(rootsubpieceFrame[['is']],how='outer', on='index').merge(lemmaFrame[['is']],how='outer',on='index')
all_entity_identity_frame.columns = ['is_root', 'is_root_subpiece','is_lemma']

# rootFrame contains the permitted parents for subroots.
# print(rootFrame)
rootFrame['is']='root'
root_parent = rootFrame.copy()
#next the frame of permitted parents for lemmas is created by combining/overlaying roots and subroots
rootsubpieceFrame['is'] = 'root_subpiece'
any_roots = root_parent.combine_first(rootsubpieceFrame)

#finally create the permitted parents of paragraphs (though we will want to examine any paragraphs subordinate directly to a root. These may indicate some datacleaning is needed, or may need to be removed entirely)
lemmaFrame['is'] = 'lemma'
anyEntityFrame = any_roots.combine_first(lemmaFrame)

anyEntityFrame.sample(10)

,paragraphIndex,is,text,mask,run_text_list
index,,,,,
2415,2415.0,lemma,ɓiraaɗam,"[True, False, False]","[ɓiraaɗam , (ɗam) / ɓiraaɗe (ɗe) , DFCZI CZ<..."
15314,15314.0,lemma,kuuca,"[True, False, False]","[kuuca , n. , D]"
15425,15425.0,root_subpiece,-IN-,[True],[-IN-]
870,870.0,lemma,baaliki,"[True, False, False]","[baaliki , n. , D [nayeejo]]"
4949,4949.0,root,ƊAAY-,[True],[ƊAAY-]
31161,31161.0,lemma,yeenga,"[True, False, False]","[yeenga , (nga) / jelle (ɗe) , Z [yeedirde, ..."
10442,10442.0,lemma,"hemre, hemere","[True, False, False]","[hemre, hemere , (nde) / keme (ɗe) , FCZ CZ<+>]"
14621,14621.0,root,KITIME,[True],[KITIME]
5941,5941.0,root,FAKAARE,[True],[FAKAARE]


Now these frames are set up, we can begin the routine. 

At each step, 
   >we merge the possible parents frame with the specific child frame.
   >then we forward-fill the parent portion of the frame to indicate the headship for every record.
   >- we want to 'unnest' this datastructure, and so we need to move from 'dictionary' style to 'record' style.
   >- records require all pertinent infromation be present inside the record (records are usually defined as a single row in most industries)
   >- so forward filling all possible parents will allow 'parentage' be infered by position / precedence on the page.
   >  - a lemma is the child of the nearest 'parent' root UP the page from it, regardless if a root is directly below it.
   >     - therefor forward-fill will allow these inferred relations be directly documented in each record, each row. 
   >     - SQL design says don't repeat yourself, but here we WANT to duplicate the parent value onto each child.
   >     - This allows us to safely remove a 'child' from its context on the page without loosing who it's parents are.
   >- then we merge in the 'frame' of next parent group, then child group, and repeat the forward filling 
   > 
   > *the process is iterative/recursive.*

However, the parent/child relationships happen to align such that each next lower group of parents is composed of the parents + the child from the previous iteration...
As a result we actually have a much simpler structure: we only need to merge each parent grouping directly, and forward fill all of those

We will have some extra duplication from top parents being copied down on each level between them and their children, but we will solve this after we create it.

The extra reason to group it this way is it does allow for a root to be declared on the same paragraph/line as a lemma, which is fairly common from manual review of the dictionary word document

In [35]:
#subroot-parent columns -> merged with <- lemma-parents columns
entityInheritanceFrame = root_parent.merge(any_roots, on='index',how ='outer',suffixes=['_main_root',''],sort=True, validate='one_to_one')

#'subroot-parent' and 'lemma-parent' columns -> merged with <- 'paragraph-parents' columns
entityInheritanceFrame = entityInheritanceFrame.merge(anyEntityFrame, on='index',how ='outer',suffixes=['_any_root','_any_entity'],sort=True, validate='one_to_one')

#before forward-fill, but we want to remove "non-paragraph" rows from our data (if present), since we don't care about empty paragraphs as children to our 'parents' (remembering that empty paragraphs are all np.nan)
#this should have no np.nan values in the final paragraph-parents column, but lets check before we accidentally hide the evidence
# print(entityInheritanceFrame.shape) >>> (15266, 12)
entityInheritanceFrame = entityInheritanceFrame.dropna(subset=['paragraphIndex_any_entity'])
# print(entityInheritanceFrame.shape) >>> (15266, 12)

#forward-fill np.nan values in the whole dataframe
entityInheritanceFrame = entityInheritanceFrame.fillna(method = 'ffill') #this can be skipped here, as it is redundant, but its a valuable midway point, and informative to review and check our assumptions

#'subroot-parent' and 'lemma-parent' and 'paragraph-parents' columns -> merged with <- 'paragraphs' columns
#note the 'paragraphs' paraFrame contains valid and invalid paragraphs, like the cleaner-output frame we dealt with above.
#it specifically only contains the indexes of paragraphs that are valid AND did not contain on of the entities we've been dealing with (roots, subroots, lemmas)
nan_free_paraFrame = nonentityParaFrame.dropna() #this will remove all non-paragraph records
allParagraphsInheritanceFrame = entityInheritanceFrame.merge(nan_free_paraFrame, on='index',how ='outer',suffixes=['','_paragraphs'],sort=True, validate='one_to_one')                                       
#since we dropped np.nan values above, the only nan values should be introduced by the paragraph merge. Where present in the paragraph columns, these will be the declaration columns of the lemmas and roots.
#to solve that we will forwardfill all columns BUT the new paragraphIndex column, which will fill in the parents for the new paragraphs.
allParagraphsInheritanceFrame.loc[:,entityInheritanceFrame.columns] = allParagraphsInheritanceFrame.loc[:,entityInheritanceFrame.columns].fillna(method = 'ffill')

#here, we can choose to dropna in the dataframe, which will remove all records that do not have a child paragraph in them.
#this will be necessary for creating records to match to the wordnet, since the paragraphs will be our glosses we need.
#however, any record that has a lemma, but does not have paragraphs still has valuable information, so we don't want to discard these completely, we just can't use them directly to search the wordnet.

# identify_condition = partial(lambda x: bool(re.match('is_',x)))
# subroot_condition = partial(lambda x: bool(re.search('_root_subpiece',x)) or bool(re.search('_anyroot',x)))

# entityInheritanceFrame
allParagraphsInheritanceFrame
#TODO break oneliners out, break roots out of those, check numbers with other checks

,paragraphIndex_main_root,is_main_root,text_main_root,mask_main_root,run_text_list_main_root,paragraphIndex_any_root,is_any_root,text_any_root,mask_any_root,run_text_list_any_root,paragraphIndex_any_entity,is_any_entity,text_any_entity,mask_any_entity,run_text_list_any_entity,paragraphIndex
index,,,,,,,,,,,,,,,,
4,4.0,root,A,[True],[A],4.0,root,A,[True],[A],4.0,root,A,[True],[A],NaN
5,4.0,root,A,[True],[A],4.0,root,A,[True],[A],5.0,lemma,a,"[True, False, False]","[a , prn,sbj,sf , DFZH Z<->]",NaN
6,4.0,root,A,[True],[A],4.0,root,A,[True],[A],5.0,lemma,a,"[True, False, False]","[a , prn,sbj,sf , DFZH Z<->]",6.0
7,4.0,root,A,[True],[A],4.0,root,A,[True],[A],5.0,lemma,a,"[True, False, False]","[a , prn,sbj,sf , DFZH Z<->]",7.0
8,4.0,root,A,[True],[A],4.0,root,A,[True],[A],8.0,lemma,-a,"[True, False, False]","[-a , suf,pos , F]",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32502,32499.0,root,ƳUUƳ-,[True],[ƳUUƳ-],32499.0,root,ƳUUƳ-,[True],[ƳUUƳ-],32500.0,lemma,ƴuuƴude,"[True, False, False]","[ƴuuƴude , v.av , D]",32502.0
32503,32503.0,root,ƳUUƳ- (2?),[True],[ƳUUƳ- (2?)],32503.0,root,ƳUUƳ- (2?),[True],[ƳUUƳ- (2?)],32503.0,root,ƳUUƳ- (2?),[True],[ƳUUƳ- (2?)],NaN
32504,32503.0,root,ƳUUƳ- (2?),[True],[ƳUUƳ- (2?)],32503.0,root,ƳUUƳ- (2?),[True],[ƳUUƳ- (2?)],32504.0,lemma,ƴuuƴude,"[True, False, False]","[ƴuuƴude , v.av , D]",NaN


In [36]:
print(allParagraphsInheritanceFrame.columns)

Index(['paragraphIndex_main_root', 'is_main_root', 'text_main_root',
       'mask_main_root', 'run_text_list_main_root', 'paragraphIndex_any_root',
       'is_any_root', 'text_any_root', 'mask_any_root',
       'run_text_list_any_root', 'paragraphIndex_any_entity', 'is_any_entity',
       'text_any_entity', 'mask_any_entity', 'run_text_list_any_entity',
       'paragraphIndex'],
      dtype='object')


In [37]:
entityDeclarationParas = allParagraphsInheritanceFrame[allParagraphsInheritanceFrame['paragraphIndex'].isna()]
entityDeclarationParasLEN = entityDeclarationParas.shape[0]
print('length of entity declaration Frame: ',entityDeclarationParasLEN)
summedEntityFramesLEN = rootFrame.shape[0]+rootsubpieceFrame.shape[0]+lemmaFrame.shape[0]
print('summed length of all 3 entity frames: ',summedEntityFramesLEN)
print('difference between the two: ',summedEntityFramesLEN-entityDeclarationParasLEN)
print('intersection of root-lemma frames: ',len(rootFrame.index.intersection(lemmaFrame.index)))
# >>>intersection of root-lemma frames:  1438
print('intersection of subroot-lemma frames: ',len(rootsubpieceFrame.index.intersection(lemmaFrame.index)))
# >>>intersection of subroot-lemma frames:  0
print('intersection of root-subroot frames: ',len(rootFrame.index.intersection(rootsubpieceFrame.index)))
# >>>intersection of root-subroot frames:  0
paragraphRecordsFrame = allParagraphsInheritanceFrame[~allParagraphsInheritanceFrame['paragraphIndex'].isna()]
print('number of content (non-entity) paragraphs: ',paragraphRecordsFrame.shape[0])
#Conclusion: entityDeclarationParas is correct from our intuition.

length of entity declaration Frame:  15266
summed length of all 3 entity frames:  16704
difference between the two:  1438
intersection of root-lemma frames:  1438
intersection of subroot-lemma frames:  0
intersection of root-subroot frames:  0
number of content (non-entity) paragraphs:  16767


In [46]:
print('unique parent types for paragraphs: ',paragraphRecordsFrame['is_any_entity'].unique(),'  ...  total found: ',len(paragraphRecordsFrame['is_any_entity'].unique()),'\n')
print(paragraphRecordsFrame.groupby('is_any_entity')['paragraphIndex'].count())
# irregular_rootINDs = list(paragraphRecordsFrame[paragraphRecordsFrame['is_any_entity']=='root'].index)



# paratext_lookup = {k:v.interogate__para_text() for k,v in parsed_object_lookup.items()}
# all_para_text_frame = pd.Series(paratext_lookup).to_frame()
# all_para_text_frame = expand_dataframe(all_para_text_frame,doc_para_count)
# spread = 10
# window_of_inds = [list(range(i-spread,i+spread)) for i in irregular_rootINDs]
# # all_para_text_frame.loc[window_of_inds[0]]


# print('\n')
# objects_list = [parsed_object_lookup[i] for i in irregular_rootINDs]
# print(objects_list[0])
# print(objects_list[0].cleaner())
# print(objects_list[0])
# print('\n')

# for o in objects_list:
#    print(o,'\n')

# savetup = (paragraphRecordsFrame[paragraphRecordsFrame['is_any_entity']=='root'],objects_list)
# with open('all_white_paras.pkl', 'wb') as file:
#    pickle.dump(savetup, file)

###Output of the above code### This was resolved by correcting the cleaner function within the pydlidocx script. It was not correctly raising the boolean success parameter. Fixed this and added a logger entry for this.
# ['lemma' 'root']
# is_any_entity
# lemma    16770
# root         4
# Name: paragraphIndex, dtype: int64
# [1982, 1983, 9318, 20565]


# run_text=['\t\t\t\t\t\t'] run_font_name=['Helv 8pt'] para_first_line_indent=None run_italic=[None] para_left_indent=None paragraph_enumeration=1982 run_font_size_pt=[8.0] run_bold=[None] para_text='\t\t\t\t\t\t'
# True
# run_text=['\t\t\t\t\t\t'] run_font_name=['Helv 8pt'] para_first_line_indent=None run_italic=[None] para_left_indent=None paragraph_enumeration=1982 run_font_size_pt=[8.0] run_bold=[None] para_text='\t\t\t\t\t\t'


# run_text=['\t\t\t\t\t\t'] run_font_name=['Helv 8pt'] para_first_line_indent=None run_italic=[None] para_left_indent=None paragraph_enumeration=1982 run_font_size_pt=[8.0] run_bold=[None] para_text='\t\t\t\t\t\t' 

# run_text=['\t\t\t\t\t\t\t\t'] run_font_name=['TmsRmn 10pt'] para_first_line_indent=None run_italic=[None] para_left_indent=None paragraph_enumeration=1983 run_font_size_pt=[None] run_bold=[None] para_text='\t\t\t\t\t\t\t\t' 

# run_text=['\t'] run_font_name=['TmsRmn 10pt'] para_first_line_indent=None run_italic=[True] para_left_indent=None paragraph_enumeration=9318 run_font_size_pt=[None] run_bold=[None] para_text='\t' 

# run_text=['\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t'] run_font_name=['TmsRmn 8pt'] para_first_line_indent=None run_italic=[None] para_left_indent=None paragraph_enumeration=20565 run_font_size_pt=[8.0] run_bold=[None] para_text='\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t' 

unique parent types for paragraphs:  ['lemma']   ...  total found:  1 

is_any_entity
lemma    16767
Name: paragraphIndex, dtype: int64
[]


In [56]:
paragraphRecordsFrame

,paragraphIndex_main_root,is_main_root,text_main_root,mask_main_root,run_text_list_main_root,paragraphIndex_any_root,is_any_root,text_any_root,mask_any_root,run_text_list_any_root,paragraphIndex_any_entity,is_any_entity,text_any_entity,mask_any_entity,run_text_list_any_entity,paragraphIndex
index,,,,,,,,,,,,,,,,
6,4.0,root,A,[True],[A],4.0,root,A,[True],[A],5.0,lemma,a,"[True, False, False]","[a , prn,sbj,sf , DFZH Z<->]",6.0
7,4.0,root,A,[True],[A],4.0,root,A,[True],[A],5.0,lemma,a,"[True, False, False]","[a , prn,sbj,sf , DFZH Z<->]",7.0
9,4.0,root,A,[True],[A],4.0,root,A,[True],[A],8.0,lemma,-a,"[True, False, False]","[-a , suf,pos , F]",9.0
10,4.0,root,A,[True],[A],4.0,root,A,[True],[A],8.0,lemma,-a,"[True, False, False]","[-a , suf,pos , F]",10.0
12,4.0,root,A,[True],[A],4.0,root,A,[True],[A],11.0,lemma,aan,"[True, False, False]","[aan , prn,ind , DFZH [an]:Z<->]",12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32498,32495.0,root,ƳUUW-,[True],[ƳUUW-],32495.0,root,ƳUUW-,[True],[ƳUUW-],32496.0,lemma,ƴuuwaade,"[True, False, False]","[ƴuuwaade , v.mv , D [ƴuubaade, ƴooraade]]",32498.0
32501,32499.0,root,ƳUUƳ-,[True],[ƳUUƳ-],32499.0,root,ƳUUƳ-,[True],[ƳUUƳ-],32500.0,lemma,ƴuuƴude,"[True, False, False]","[ƴuuƴude , v.av , D]",32501.0
32502,32499.0,root,ƳUUƳ-,[True],[ƳUUƳ-],32499.0,root,ƳUUƳ-,[True],[ƳUUƳ-],32500.0,lemma,ƴuuƴude,"[True, False, False]","[ƴuuƴude , v.av , D]",32502.0


In [99]:
paragraphRecordsFrame.columns
cols = [
   'paragraphIndex_main_root', 
   'is_main_root', 
   'text_main_root',
   'mask_main_root', 
   'run_text_list_main_root', 
   'paragraphIndex_any_root',
   'is_any_root', 
   'text_any_root', 
   'mask_any_root',
   'run_text_list_any_root', 
   'paragraphIndex_any_entity', 
   'is_any_entity',
   'text_any_entity', 
   'mask_any_entity', 
   'run_text_list_any_entity',
   'paragraphIndex']

expunge_root_from_subroot = [   
   'paragraphIndex_any_root',
   'is_any_root', 
   'text_any_root', 
   'mask_any_root',
   'run_text_list_any_root', 
   # 'paragraphIndex_any_entity', 
   # 'is_any_entity',
   # 'text_any_entity', 
   # 'mask_any_entity', 
   # 'run_text_list_any_entity',
   ]
drop_cols = [
   # 'paragraphIndex_main_root', 
   'is_main_root', 
   'text_main_root',
   'mask_main_root', 
   # 'run_text_list_main_root', 
   # 'paragraphIndex_any_root',
   'is_any_root', 
   'text_any_root', 
   'mask_any_root',
   # 'run_text_list_any_root', 
   # 'paragraphIndex_any_entity', 
   'is_any_entity',
   # 'text_any_entity', 
   # 'mask_any_entity', 
   # 'run_text_list_any_entity',
   # 'paragraphIndex'
   ]

colsnewnames = [
   'paragraphIndex_root', 
   # 'is_main_root', 
   # 'text_main_root',
   # 'mask_main_root', 
   'run_text_list_root', 
   'paragraphIndex_root_subpiece',
   # 'is_any_root', 
   # 'text_any_root', 
   # 'mask_any_root',
   'run_text_list_root_subpiece', 
   'paragraphIndex_lemma', 
   # 'is_any_entity',
   'text_lemma', 
   'mask_lemma', 
   'run_text_list_lemma',
   'paragraphIndex_paragraph',
   'run_text_list_paragraph'
   ]
paragraphRecordsFrame['run_text_list_paragraph'] = paragraphRecordsFrame['paragraphIndex'].apply(lambda x: parsed_object_lookup[x].run_text)

test_frame = paragraphRecordsFrame.copy(deep=True)
test_frame.loc[test_frame['is_any_root']=='root',expunge_root_from_subroot] = np.nan
print(test_frame.columns)
test_frame.drop(drop_cols, axis = 1, inplace=True)
test_frame.columns = colsnewnames
test_frame

Index(['paragraphIndex_main_root', 'is_main_root', 'text_main_root',
       'mask_main_root', 'run_text_list_main_root', 'paragraphIndex_any_root',
       'is_any_root', 'text_any_root', 'mask_any_root',
       'run_text_list_any_root', 'paragraphIndex_any_entity', 'is_any_entity',
       'text_any_entity', 'mask_any_entity', 'run_text_list_any_entity',
       'paragraphIndex', 'run_text_list_paragraph'],
      dtype='object')


c:\Users\peter\VSC-Workspace_PULAR\.pular_venv\lib\site-packages\ipykernel_launcher.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,paragraphIndex_root,run_text_list_root,paragraphIndex_root_subpiece,run_text_list_root_subpiece,paragraphIndex_lemma,text_lemma,mask_lemma,run_text_list_lemma,paragraphIndex_paragraph,run_text_list_paragraph
index,,,,,,,,,,
6,4.0,[A],NaN,NaN,5.0,a,"[True, False, False]","[a , prn,sbj,sf , DFZH Z<->]",6.0,[you (sg.)]
7,4.0,[A],NaN,NaN,5.0,a,"[True, False, False]","[a , prn,sbj,sf , DFZH Z<->]",7.0,[tu]
9,4.0,[A],NaN,NaN,8.0,-a,"[True, False, False]","[-a , suf,pos , F]",9.0,[your (sg.) (only with certain nouns such as t...
10,4.0,[A],NaN,NaN,8.0,-a,"[True, False, False]","[-a , suf,pos , F]",10.0,"[ton, ta, tes (seulement pour certains substan..."
12,4.0,[A],NaN,NaN,11.0,aan,"[True, False, False]","[aan , prn,ind , DFZH [an]:Z<->]",12.0,[you (sg.) (emphatic)]
...,...,...,...,...,...,...,...,...,...,...
32498,32495.0,[ƳUUW-],NaN,NaN,32496.0,ƴuuwaade,"[True, False, False]","[ƴuuwaade , v.mv , D [ƴuubaade, ƴooraade]]",32498.0,[s'incliner; se cabrer]
32501,32499.0,[ƳUUƳ-],NaN,NaN,32500.0,ƴuuƴude,"[True, False, False]","[ƴuuƴude , v.av , D]",32501.0,[to increase in volume]
32502,32499.0,[ƳUUƳ-],NaN,NaN,32500.0,ƴuuƴude,"[True, False, False]","[ƴuuƴude , v.av , D]",32502.0,[augmenter de volume]


In [84]:
test_frame['run_text_list_paragraph'] = test_frame['paragraphIndex_paragraph'].apply(lambda x: parsed_object_lookup[x].run_text)
test_frame

,paragraphIndex_root,run_text_list_root,paragraphIndex_root_subpiece,run_text_list_root_subpiece,paragraphIndex_lemma,text_lemma,mask_lemma,run_text_list_lemma,paragraphIndex_paragraph,run_text_list_paragraph
index,,,,,,,,,,
6,4.0,[A],NaN,NaN,5.0,a,"[True, False, False]","[a , prn,sbj,sf , DFZH Z<->]",6.0,[you (sg.)]
7,4.0,[A],NaN,NaN,5.0,a,"[True, False, False]","[a , prn,sbj,sf , DFZH Z<->]",7.0,[tu]
9,4.0,[A],NaN,NaN,8.0,-a,"[True, False, False]","[-a , suf,pos , F]",9.0,[your (sg.) (only with certain nouns such as t...
10,4.0,[A],NaN,NaN,8.0,-a,"[True, False, False]","[-a , suf,pos , F]",10.0,"[ton, ta, tes (seulement pour certains substan..."
12,4.0,[A],NaN,NaN,11.0,aan,"[True, False, False]","[aan , prn,ind , DFZH [an]:Z<->]",12.0,[you (sg.) (emphatic)]
...,...,...,...,...,...,...,...,...,...,...
32498,32495.0,[ƳUUW-],NaN,NaN,32496.0,ƴuuwaade,"[True, False, False]","[ƴuuwaade , v.mv , D [ƴuubaade, ƴooraade]]",32498.0,[s'incliner; se cabrer]
32501,32499.0,[ƳUUƳ-],NaN,NaN,32500.0,ƴuuƴude,"[True, False, False]","[ƴuuƴude , v.av , D]",32501.0,[to increase in volume]
32502,32499.0,[ƳUUƳ-],NaN,NaN,32500.0,ƴuuƴude,"[True, False, False]","[ƴuuƴude , v.av , D]",32502.0,[augmenter de volume]


In [76]:
test_frame2 = test_frame['mask_lemma'].apply(pd.Series)
test_frame2

,0,1,2,3,4
index,,,,,
6,True,False,False,NaN,NaN
7,True,False,False,NaN,NaN
9,True,False,False,NaN,NaN
10,True,False,False,NaN,NaN
12,True,False,False,NaN,NaN
...,...,...,...,...,...
32498,True,False,False,NaN,NaN
32501,True,False,False,NaN,NaN
32502,True,False,False,NaN,NaN


In [117]:
pd.set_option('display.max_rows', 200)
print(parsed_object_lookup[24476])
# print(paragraphRecordsFrame.loc[24474,:])

# print(test_frame2[test_frame2[0]==False])
test_frame.loc[24476]
bold_where_not_expected = test_frame2[test_frame2[[1,2,3,4]].isin([True]).any(axis=1)] #94, only 2 dont

bold_out_of_place = test_frame2[test_frame2[0]!=True] #only 2 arent in rows with lemmas 24476, 29842
bold_where_not_expected
# test_frame.loc[bold_where_not_expected.index]
bold_out_of_place
# test_frame.loc[24476]
# test_frame.loc[29842]
# paragraphRecordsFrame[paragraphRecordsFrame.isin(['SUK- (2?)']).any(axis=1)]
# paragraphRecordsFrame.loc[25747] #additional lemmas are being claimed that aren't lemmas

run_italic=[None] run_bold=[None] para_first_line_indent=None run_font_name=['TmsRmn 8pt'] para_text='ceertugol (Math.)(T): disjunction ; disjonction' paragraph_enumeration=24476 run_font_size_pt=[8.0] run_text=['ceertugol (Math.)(T): disjunction ; disjonction'] para_left_indent=116128800


,0,1,2,3,4
index,,,,,
24476,False,True,False,NaN,NaN
29842,False,True,False,NaN,NaN


In [114]:
allentitymasks = entityDeclarationParas['mask_any_entity'].apply(pd.Series)

In [115]:
entity_out_of_place = allentitymasks[allentitymasks[0]!=True]

In [126]:
entity_out_of_place.loc[:,'paragraph_run_text'] = entity_out_of_place.index.values
entity_out_of_place.loc[:,'paragraph_run_text'] = entity_out_of_place.loc[:,'paragraph_run_text'].apply(lambda x: parsed_object_lookup[x].run_text)
entity_out_of_place

c:\Users\peter\VSC-Workspace_PULAR\.pular_venv\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


,0,1,2,3,4,5,6,7,paragraph_run_text
index,,,,,,,,,
4644,False,True,False,NaN,NaN,NaN,NaN,NaN,"[to hit hard on; , (neɗɗo), to beat sb; to be..."
4645,False,True,False,NaN,NaN,NaN,NaN,NaN,"[taper fort sur(D); , (neɗɗo), battre qqn, ta..."
4675,False,True,False,NaN,NaN,NaN,NaN,NaN,"[to get angry at sb, scold sb; to complain; to..."
4676,False,True,False,NaN,NaN,NaN,NaN,NaN,"[se fâcher à qqn, gronder qqn; se plaindre; ba..."
5980,False,True,False,NaN,NaN,NaN,NaN,NaN,"[to put o.s. across; , (neɗɗo), to be an obst..."
5981,False,True,False,NaN,NaN,NaN,NaN,NaN,"[se mettre de travers(D); , (neɗɗo), être obs..."
6081,False,True,False,NaN,NaN,NaN,NaN,NaN,"[to make small (intensive); , (neɗɗo), to hum..."
6082,False,True,False,NaN,NaN,NaN,NaN,NaN,"[rendre petit (intensif); , (neɗɗo), rabaisse..."
6681,False,False,False,False,False,True,False,NaN,[representative of the So or Sidibe clan; warr...


In [127]:
entity_out_of_place_run_text_exploded = entity_out_of_place['paragraph_run_text'].apply(pd.Series)
entity_out_of_place_run_text_exploded

,0,1,2,3,4,5,6
index,,,,,,,
4644,to hit hard on;,(neɗɗo),to beat sb; to beat (heart),NaN,NaN,NaN,NaN
4645,taper fort sur(D);,(neɗɗo),"battre qqn, taper qqn (Z); battre (coeur)(Z)",NaN,NaN,NaN,NaN
4675,"to get angry at sb, scold sb; to complain; to ...",(e neɗɗo),to complain to sb(F),NaN,NaN,NaN,NaN
4676,"se fâcher à qqn, gronder qqn; se plaindre; bav...",(e neɗɗo),se plaindre à qqn,NaN,NaN,NaN,NaN
5980,to put o.s. across;,(neɗɗo),"to be an obstacle to sb, get in the way, bar ...",NaN,NaN,NaN,NaN
5981,se mettre de travers(D);,(neɗɗo),"être obstacle pour qqn, entraver, barrer la r...",NaN,NaN,NaN,NaN
6081,to make small (intensive);,(neɗɗo),"to humble, humiliate sb",NaN,NaN,NaN,NaN
6082,rendre petit (intensif);,(neɗɗo),"rabaisser, humilier qqn",NaN,NaN,NaN,NaN
6681,representative of the So or Sidibe clan; warri...,arɗo,for the Sidibe or So clan;,(pl.),those who emigrate;,Ferooɓe,- clan Sidibe or So [Soosooɓe]


In [ ]:
# next steps, force entities to have a starting run with the value. "match" for the feature, not search
#repeat cleaning, groupby lemma index, begin work on dictionary extraction with what I have. Record metrics of the values being filtered.
# some ongoing attrition score

In [ ]:
pd.reset_option("^display")

In [3]:
outputs = [
   
   'txt_FulaLemmas',
   'txt_FulaRoots',
   
   #extra
   'txt_FulaAnnotations',
   #gloss derivatives
   'txt_FulaSenseEnglish',
   'txt_FulaSenseFrench',
   'txt_FulaSenseClassifications',
   'txt_FulaSenseEnglishAnnotations',
   'txt_FulaSenseFrenchAnnotations',

   #lemma derivative
   'txt_FulaDialects',
   'txt_FulaPOSTags',
   'txt_FulaSynonyms',


   #agg
   'txt_FulaInParenthesis',
   'txt_POS',
   'txt_RootOrigins'
]

In [4]:
# noun_patterns = [r'n\.',r"(?<=\()[^\)]+",r"\/"]
# nounPatternRegex = re.compile('|'.join([p for p in noun_patterns]))
# print(nounPatternRegex)
# s = 'n. dfhjkgqh (wsh) askldjhf / asdfhjkkh'
# print(re.findall(nounPatternRegex,s))
# print(nounPatternRegex.findall(s))

# if /, the next word is a plural. The plural also then usually has a (el) class after it. commas are multiple version
# so each lemma noun may have a (class). The lemma may have one or multiple plurals, which will have a modified (class)
# so have a nouns list? nouns may then be 
# [('n.',{bool}), #only needed if this gets added to all POS? Or maybe theres information in its use?
# ([[{word}],{Optional[class]}]),  #singular
   #([[{word}],{Optional[class]}])]   #plural

# so nounStruct[0][1] is the flag for noun?
   # one POS table, and noun will get added there. But the presence of a noun indicates to look here
# classes may be found [sClass[1] for sClass in nounStruct[1]] will give all classes of singular, [sClass[1] for sClass in nounStruct[1]]

re.compile('n\\.|(?<=\\()[^\\)]+|\\/')
['n.', 'wsh', '/']
['n.', 'wsh', '/']


In [5]:
# now = datetime.now()
# current_time = now.strftime("%Y-%m-%d_-_%H-%M-%S")
# print(f"Experiment time: {current_time}\nExperiment note: {experiment}\n\n")

In [30]:
def getAssert(dct,val,typ:type, strict = True) -> Any:
   out = dct.get(val,None)
   if strict and typ is type(None):
      raise NotImplementedError(f'this get/assert function cannot "get" None values safely, and a {val} type was passed for dict {dict}')
   if strict and out is None:
      raise KeyError(f"the provided dict {dct} did not have the key: {val}")
   assert isinstance(out,typ), f"the provided dict {dct} did not give {val} with the expected type: {typ}, but instead {out}"
   return out
   
def trustyGet(obj:Docx_Paragraph_and_Runs, feat: str, silent_return = True) -> str:
      if not silent_return:
         output : str = getattr(obj,feat,'')
         if len(output) > 0:
            return output
         else:
            raise AttributeError(f'trusty getter could not find: {feat}')
      else:
         output : str = getattr(obj,feat,'')
         return output

In [36]:
class dict_entry(pydantic.BaseModel): 
   '''
   #True is used as default value, and must be changed to False if the feature is not found. This will catch not implemented, and incomplete updates
   '''
   irregularities: List[str] = []
   lemma_index: int 
   paragraphs: List[Docx_Paragraph_and_Runs] = Field(...,min_items = 1)
   lemma: str = Field(...,min_length = 1) #article
   root: str
   root_subpiece: Union[str,bool] = True #these will not be updated after initialization
   root_metadata: Dict[str,Any]  #index,root_text, root_meta_data #article 
   root_origin: Union[str,bool] = True #these will not be updated after initialization
   #Paragraph Derivatives: Glosses and Annotations
   lemmaLine_runs: List[str] #these will not be updated after initialization
   englishGlossLine_runs: Union[List[str],bool] = True #these will not be updated after initialization
   frenchGlossLine_runs: Union[List[str],bool] = True #these will not be updated after initialization
   FulaAnnotations_runs: Union[List[List[str]],bool]= True #these will not be updated after initialization

   #Gloss Derivatives: English Senses
   FulaSenseEnglish: Union[List[str],bool] = True #list of "senses" split by semicolons #article
   FulaSenseEnglish_Count: Union[int,bool] = True #number of senses in english #aka FulaSenseClassifications #article
   FulaSenseEnglish_Annotations: Union[List[str],bool] = True #contains the annotations (in parenthesis) for a given sense, if any #article 
   FulaSenseEnglish_Synonyms: Union[List[str],bool] = True #holds bracket text for a sense, suspected all synonyms. These may all occur at the end, and may be redundant with the synonyms provided at the head of the entry
   FulaSenseEnglish_unusedContent: Union[List[str],bool] = True #holds any run text that is not contained in the above features
   
   #Gloss Derivatives: French Senses
   FulaSenseFrench: Union[List[str],bool] = True #article
   FulaSenseFrench_Count: Union[int,bool] = True #aka FulaSenseClassifications
   FulaSenseFrench_Annotations: Union[List[str],bool] = True
   FulaSenseFrench_Synonyms: Union[List[str],bool] = True
   FulaSenseFrench_unusedContent: Union[List[str],bool] = True
   
   #lemma derivative
   lemmaLine_unusedContent: List[Tuple[int,str]] #= Field(...) #int is index for run to allow tracing as entries are removed
   FulaDialects: Union[List[str],bool] = True #article
   FulaPOSTags: Union[List[str],bool] = True #article
   FulaPOSClass: Union[List[str],bool] = True #Noun, Adj, Verb, Adv, Prn, ..., Complicated, Indeterminate
   FulaNoun_NounsAndClass: Union[List[Tuple[str,str]],bool] = True #Pular has unique noun classes. Lemma will be copied here beside its class ("noun", "nounclass"), and any additional singular forms will be in additional tuples in this same list
   FulaNoun_PluralsAndClass: Union[List[Tuple[str,str]],bool] = True #Dict provides plurals for nouns. Tuples will have ("noun", "nounclass")
   FulaSynonyms: Union[List[str],bool] = True #article
   FulaCrossRef: Union[List[str],bool] = True
         # FulaVerbClass:
   
   class Config:
      validate_all = True
      validate_assignment = True
      smart_union = True  
      extra = 'forbid'

   @root_validator(pre=True)
   def _validate_and_build(cls, values: Dict[str,Any]) -> Dict[str, Any]:
      # print(values)
      newValues: Dict[str,Any] = {}
      newValues['irregularities'] = []

      ###PARAGRAPHS CHECK###
      print(values)
      newValues['paragraphs'] = getAssert(values,'paragraphs',list) #these come in as tuples(ind,para)
      #TODO have more rigorous checks on para since so much depends on that even in this validation
      assert isinstance(newValues['paragraphs'][0],tuple), 'currently paras are packaged in a tuple with their original doc index'
      newValues['paragraphs'] = [p for i,p in newValues['paragraphs']]
      if len(newValues['paragraphs']) == 0:
         raise ValueError('given invalid paragraphs of zero length')
      else: #logic for subsidary paragraphs
         lemma_line = trustyGet(newValues['paragraphs'][0],feat = 'run_text', silent_return=False)
         newValues['lemmaLine_runs'] = lemma_line
         if len(newValues['paragraphs']) == 1:
            newValues['englishGlossLine_runs'] = False
            newValues['frenchGlossLine_runs'] = False
         elif len(newValues['paragraphs']) == 2:
            newValues['irregularities'].append('What:ambiguous, Where:paragraphs, Why: only one')
            newValues['englishGlossLine_runs'] = False
            newValues['frenchGlossLine_runs'] = False
         elif len(newValues['paragraphs']) == 3:
            newValues['englishGlossLine_runs'] = newValues['paragraphs'][1].get_run_text()
            newValues['frenchGlossLine_runs'] = newValues['paragraphs'][2].get_run_text()
         else: 
            newValues['englishGlossLine_runs'] = newValues['paragraphs'][1].get_run_text()
            newValues['frenchGlossLine_runs'] = newValues['paragraphs'][2].get_run_text()
            newValues['FulaAnnotations_runs'] = newValues['paragraphs'][3:].get_run_text()
      
      
      try: ###LEMMA CHECK###
         #reading in lemma results from first pass of pydantic parser
         lemma_index, lemma_mask, lemmaLine_runs = getAssert(values,'lemma',tuple)
         #checking for correct structure
         lemma_matched_runs = list(compress(lemmaLine_runs,lemma_mask))
         if len(lemma_matched_runs) > 1:
            newValues['irregularities'].append('What:Unexpected, Where: Lemmas, Why: the merge routines should aggregate adjacent runs with same features. Multiple Lemma runs should not be possible if Bold is contiguous')
         lemma_text = ''.join(chain(lemma_matched_runs)).strip()
            #TODO have better control for expected structure that these runs should be adjacent (and only should be one)
         #saving values. These will have to pass type check of declared attribute types for validation to succeed
         newValues['lemma'] = lemma_text
         newValues['lemma_index'] = lemma_index
         newValues['lemmaLine_runs'] = lemmaLine_runs
         used_run_mask = lemma_mask
      except Exception as e:
         #exception lemma check
         raise RuntimeError('failed lemma check') from e

      try: ###ROOT CHECK###
         #reading in root results from first pass of pydantic parser
         #checking for correct structure
         root_index, root_mask, rootLine_runs = getAssert(values,'root',tuple)
         root_matched_runs = list(compress(rootLine_runs,root_mask))
         if len(root_matched_runs) > 1:
            newValues['irregularities'].append('What:Unexpected, Where: Root, Why: the merge routines should aggregate adjacent runs with same features. Multiple Root runs should not be possible if Fontsize is contiguous and unique')
         root_text = ''.join(chain(root_matched_runs)).strip()
         if root_index == newValues['lemma_index']:
            newValues['irregularities'].append('What:inconsistent, Where: Lemmas and Root, Why:normally root and lemma are on different lines. This has them sharing, which may indicate a lack of other content')
            used_run_mask = [any(run) for run in list(zip(used_run_mask,root_mask))]
            rootLine_runs = False
         newValues['root'] = root_text
         newValues['root_metadata'] = {'root_index': root_index, 'root_runs':rootLine_runs}
      except Exception as e:
         #exception ROOT check
         raise RuntimeError('failed ROOT check') from e

      try: ###ROOT-Subpiece CHECK###
         #iterating in case a subroot is present
         root_index, root_mask, rootLine_runs = getAssert(values,'root',tuple)
         root_matched_runs = list(compress(rootLine_runs,root_mask))
         if len(root_matched_runs) > 1:
            newValues['irregularities'].append('What:Unexpected, Where: Root-Subpiece, Why: the merge routines should aggregate adjacent runs with same features. Multiple Root runs should not be possible if Fontsize is contiguous and unique')
         root_subpiece_text = ''.join(chain(root_matched_runs)).strip()
         if root_index == newValues['lemma_index']:
            newValues['irregularities'].append('What:inconsistent, Where: Lemmas and Root-Subpiece, Why:normally root and lemma are on different lines. This has them sharing, which may indicate a lack of other content')
            used_run_mask = [any(run) for run in list(zip(used_run_mask,root_mask))]
            rootLine_runs = False
         newValues['root_subpiece'] = root_subpiece_text
         newValues['root_metadata'] = {'root_subpiece_index': root_index, 'root_subpiece_runs':rootLine_runs}
      except AssertionError as e:
         newValues['root_subpiece'] = False
      except Exception as e:
         #exception ROOT-Subpiece check
         raise RuntimeError('failed ROOT-Subpiece check') from e

      #determining lemmaLine_unusedContent
      unused_run_mask = [not r for r in used_run_mask]
      print(list(compress(enumerate(newValues['lemmaLine_runs']),unused_run_mask)))
      newValues['lemmaLine_unusedContent'] = list(compress(enumerate(newValues['lemmaLine_runs']),unused_run_mask))
      
      return newValues

   def parse_senses(self):
      #TODO
      return

   def parse_glossRemainder(self):
      #TODO
      return

   def parse_lemmaLine(self):
      #TODO
      return

   def parse_lemmaLineRemainder(self):
      #TODO
      return

   def get_rootOrigin(self):
      #TODO
      return
 
   def give_entryText(self, joiner = '\t') -> str: #article
      return joiner.join([para.trustyGet('para_text') for para in self.paragraphs])

   
      
            
# dict_entry.parse_obj({'paragraphs':[b for a,b in parsed_object_list[5:8]]})

# incoming = {
#    'paragraphs': List[Docx_Paragraph_and_Runs],
#    'root': List[int,str,List[bool]],
#    'lemma': List[int,str,List[bool]],
#    # 'pos': 
# }

# [
#    ['paras'], #obj
#    ['parsed_paras'], #pydocx
#    ['roots'], #name, fill forwards
#    ['lemmas']  #text, fill forwards
# ]

In [8]:
# import pickle
# with open('parsed_objectClass_outcomes_dict.pkl', 'rb') as file:
#     # Call load method to deserialze
#     parsed_objectClass_outcomes_dict = pickle.load(file, encoding='utf-8')

# parsed_object_list = parsed_objectClass_outcomes_dict['parsed_object_list'] 
# para_text_lookup = parsed_objectClass_outcomes_dict['para_text_lookup'] 
# root_ind_list = parsed_objectClass_outcomes_dict['root_ind_list'] 
# subroot_ind_list = parsed_objectClass_outcomes_dict['subroot_ind_list'] 
# lemma_ind_list = parsed_objectClass_outcomes_dict['lemma_ind_list'] 
# root_and_lemma_one_line = parsed_objectClass_outcomes_dict['root_and_lemma_one_line'] 
# root_lookup = parsed_objectClass_outcomes_dict['root_lookup'] 
# lemma_lookup = parsed_objectClass_outcomes_dict['lemma_lookup'] 
# # char_counts = parsed_objectClass_outcomes_dict['char_counts']

In [9]:
# docx_filename = "pasted_docx page 1.docx"
docx_filename = "Fula_Dictionary-repaired.docx"
parsed_to_dict = read_docx(docx_filename)

outcomes_dict = monolith_root_and_lemma_processor(parsed_to_dict['parsed_object_list'],parsed_to_dict['char_counts'])

parsed_object_list = outcomes_dict['parsed_object_list'] 
para_text_lookup = outcomes_dict['para_text_lookup'] 
root_ind_list = outcomes_dict['root_ind_list'] 
subroot_ind_list = outcomes_dict['subroot_ind_list'] 
lemma_ind_list = outcomes_dict['lemma_ind_list'] 
root_and_lemma_one_line = outcomes_dict['root_and_lemma_one_line'] 
root_lookup = outcomes_dict['root_lookup'] 
lemma_lookup = outcomes_dict['lemma_lookup'] 
char_counts = outcomes_dict['char_counts']

In [ ]:
# #Creating lists of indexes for use in remaining processing

# # # print(len(parsed_object_list)) #32040
# #note this parsed paras are packaged with the original para number in the docx
# #therefore 32507 is the index to use, but len of that IS 32040, since ~500 were empty paragraphs that were not parsed
# parsed_para_indexes = [i for i,p in parsed_object_list]

# lim = 32507
# # rootInds = [x for x in root_ind_list if x < lim]
# rootInds=root_ind_list
# # subRootInds = [x for x in subroot_ind_list if x < lim]
# subRootInds=subroot_ind_list
# # lemmaInds = [x for x in lemma_ind_list if x < lim]
# lemmaInds=lemma_ind_list

# allroots = rootInds.copy()
# allroots.extend(subRootInds)
# allroots = sorted(allroots)
# allEntities = allroots.copy()
# allEntities.extend(lemmaInds)
# allEntities = sorted(set(allEntities))
# print('allEntities: ',len(allEntities))


# # all_paras = [i for i,obj in parsed_object_list if i < lim]
# normal_para = parsed_para_indexes.copy()
# [normal_para.remove(i) for i in allEntities]
# print('normal_para: ',len(normal_para))

# # normal_para = [x for x in parsed_para_indexes if all([x not in rootInds, x not in lemmaInds, x not in subRootInds])]
# # print(normal_para)
# # print(allroots)
# root_aligned_lemmas = list(closest(pairwise(allroots), lemmaInds))
# lemma_aligned_paras = list(closest(pairwise(lemmaInds),normal_para))

In [39]:
# import pickle
# with open('entity_index_schema.pkl', 'rb') as file:
#     # Call load method to deserialze
#     entity_index_schema = pickle.load(file, encoding='utf-8')

In [ ]:
# for r,v1 in entity_index_schema.items():
#    root = (r,root_lookup[r][1],root_lookup[r][2])
#    if isinstance(v1,dict):
#       for k,v in v1.items():
#          if k in subroot_ind_list:
#             subroot

In [38]:
# result = []
# for i in parsed_object_list[5:15]:
#    result.append(dict_entry({i))
# print(root_lookup[4])
# print(para_text_lookup[3])
# root_container = [(ind, mask, rootLine_runs) for ind, mask, rootLine_runs in getAssert(values,'root',list)]
shorter_obj_list = parsed_object_list[:300]
# for r, l
inds = [8,9,10]

dct = {
   'paragraphs':[o for o in shorter_obj_list if o[0] in inds],
   'root': (4,root_lookup[4][1],root_lookup[4][2]),
   'lemma': (inds[0],lemma_lookup[inds[0]][1],lemma_lookup[inds[0]][2])
}
dict_entry.parse_obj(dct)

# {'paragraphs': [(9, 
#    Docx_Paragraph_and_Runs(run_italic=[None], para_left_indent=274320, run_font_name=['TmsRmn 10pt'], paragraph_enumeration=9, run_font_size_pt=[None], para_first_line_indent=-91440, run_text=['your (sg.) (only with certain nouns such as those which refer to close family members)'], run_bold=[None], para_text='your (sg.) (only with certain nouns such as those which refer to close family members)')), (10, Docx_Paragraph_and_Runs(run_italic=[None], para_left_indent=274320, run_font_name=['TmsRmn 10pt'], paragraph_enumeration=10, run_font_size_pt=[None], para_first_line_indent=-91440, run_text=['ton, ta, tes (seulement pour certains substantifs, tels ceux qui se rapportent aux membres de la proche famille)'], run_bold=[None], para_text='ton, ta, tes (seulement pour certains substantifs, tels ceux qui se rapportent aux membres de la proche famille)')), 
#    (11, Docx_Paragraph_and_Runs(run_italic=[None, True, None], para_left_indent=182880, run_font_name=['TmsRmn 10pt', 'TmsRmn 10pt', 'Helv 8pt'], paragraph_enumeration=11, run_font_size_pt=[None, None, 8.0], para_first_line_indent=-91440, run_text=['aan  ', 'prn,ind  ', 'DFZH  [an]:Z<->'], run_bold=[True, None, None], para_text='aan  prn,ind  DFZH  [an]:Z<->'))], 'root': [(4, [True], ['A'])], 'lemma': (5, [True, False, False], ['a  ', 'prn,sbj,sf  ', 'DFZH  Z<->'])
#    } did not have the key: paragraphs"

{'paragraphs': [(8, Docx_Paragraph_and_Runs(run_italic=[None, True, None], para_left_indent=182880, run_font_name=['TmsRmn 10pt', 'TmsRmn 10pt', 'Helv 8pt'], paragraph_enumeration=8, run_font_size_pt=[None, None, 8.0], para_first_line_indent=-91440, run_text=['-a  ', 'suf,pos  ', 'F'], run_bold=[True, None, None], para_text='-a  suf,pos  F')), (9, Docx_Paragraph_and_Runs(run_italic=[None], para_left_indent=274320, run_font_name=['TmsRmn 10pt'], paragraph_enumeration=9, run_font_size_pt=[None], para_first_line_indent=-91440, run_text=['your (sg.) (only with certain nouns such as those which refer to close family members)'], run_bold=[None], para_text='your (sg.) (only with certain nouns such as those which refer to close family members)')), (10, Docx_Paragraph_and_Runs(run_italic=[None], para_left_indent=274320, run_font_name=['TmsRmn 10pt'], paragraph_enumeration=10, run_font_size_pt=[None], para_first_line_indent=-91440, run_text=['ton, ta, tes (seulement pour certains substantifs, te

dict_entry(irregularities=[], lemma_index=8, paragraphs=[Docx_Paragraph_and_Runs(run_italic=[None, True, None], para_left_indent=116128800, run_font_name=['TmsRmn 10pt', 'TmsRmn 10pt', 'Helv 8pt'], paragraph_enumeration=8, run_font_size_pt=[None, None, 8.0], para_first_line_indent=-91440, run_text=['-a  ', 'suf,pos  ', 'F'], run_bold=[True, None, None], para_text='-a  suf,pos  F'), Docx_Paragraph_and_Runs(run_italic=[None], para_left_indent=174193200, run_font_name=['TmsRmn 10pt'], paragraph_enumeration=9, run_font_size_pt=[None], para_first_line_indent=-91440, run_text=['your (sg.) (only with certain nouns such as those which refer to close family members)'], run_bold=[None], para_text='your (sg.) (only with certain nouns such as those which refer to close family members)'), Docx_Paragraph_and_Runs(run_italic=[None], para_left_indent=174193200, run_font_name=['TmsRmn 10pt'], paragraph_enumeration=10, run_font_size_pt=[None], para_first_line_indent=-91440, run_text=['ton, ta, tes (seul